In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import re

/Users/zenmaster/Programming/clasificados/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [45]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [31]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [49]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')
multi_check_box_toggle.click()
time.sleep(2)

In [50]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')
select_all.click()
time.sleep(2)

In [51]:
time.sleep(10)
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')
ver_listado.click()

In [54]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [ ]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')
order_by.click()
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
ultimos_publicados.click()

In [12]:
list_cards = []

for i in range(1,16):
    table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
    table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
    table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    dictionary_cards = {}
    dictionary_cards_2 = {}
    try:
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text
        
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text

    except NoSuchElementException:
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
    try:
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')

        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')

    except NoSuchElementException:
        dictionary_cards['Broker'] = 'No Broker'
        dictionary_cards_2['Broker'] = 'No Broker'

    dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
    dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
    dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
    dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
    dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
    list_cards.append(dictionary_cards)

    dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
    dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
    dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
    dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
    dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
    list_cards.append(dictionary_cards_2)


In [ ]:
df = pd.json_normalize(list_cards)
df

In [ ]:
next_page = driver.find_element(By.XPATH,'//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
next_page.click()

In [5]:
last_modified = driver.execute_script("return document.lastModified;")

In [ ]:
last_modified

In [ ]:
import requests

# Send a GET request to the webpage
response = requests.get("https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4774562")  # Replace this URL with the URL of your webpage

# Check the Last-Modified header in the response
last_modified_header = response.headers.get("Last-Modified")

# Print the Last-Modified header
print("Last modified (from HTTP header):", last_modified_header)

In [ ]:
all_cards = []

# Infinite loop to navigate through pages
for x in range(3):
    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            list_cards.append(dictionary_cards)
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
            list_cards.append(dictionary_cards_2)
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)

# Save to CSV or process further
df.to_csv("classifieds_data.csv", index=False)

print("Data collection completed.")

In [90]:
def clean_real_estate_data(df):
    # Clean the 'Type' column by removing leading commas and whitespace
    df['Type'] = df['Type'].str.replace(r'^,\s*', '', regex=True)

    # Clean the 'Broker' column using the combined logic
    def clean_broker(broker_name):
        try:
            # Remove any text starting with " - " followed by anything after it
            broker_name = re.sub(r'\s*-\s*.*$', '', broker_name)
            
            # Remove any text starting with "#" followed by digits
            broker_name = re.sub(r'\s*#\d+', '', broker_name)
            
            # Remove any text starting with "lic." followed by digits
            broker_name = re.sub(r'\s*lic\.\s*\d+', '', broker_name)
            
        except Exception as e:
            print(f"An error occurred: {e}")

        # Return the cleaned broker name
        return broker_name.strip()
    
    # Apply the clean_broker function to the 'Broker' column
    df['Broker'] = df['Broker'].apply(clean_broker)

    # Drop the 'Title' column if needed
    # df.drop(columns=['Title'], inplace=True)

    # Extract the number of bedrooms and bathrooms from the 'Rooms' column
    df[['Bedrooms', 'Bathrooms']] = df['Rooms'].str.extract(r'(\d+)\s+Cuartos.*\|\s*(\d+)\s+Baños', expand=True)
    df[['Bedrooms', 'Bathrooms']] = df[['Bedrooms', 'Bathrooms']].fillna(0)

    df['Location'] = df['Location'].fillna('')

    # Split the 'Location' column by the comma into two columns
    df[['Neighborhood', 'City']] = df['Location'].str.split(',', n=1, expand=True)

    # Strip any leading or trailing whitespace from the new columns
    df['Neighborhood'] = df['Neighborhood'].str.strip()
    df['City'] = df['City'].str.strip()

    df['Broker'] = df['Broker'].str.replace('ClasificadosOnline', '').str.strip()

    return df

In [ ]:
# Initialize an empty list to store all data
all_cards = []

# Initialize a page counter (optional, for debugging or tracking pages)
page_number = 1

# Infinite loop to navigate through pages
for x in range(3):

    dropdown_element = driver.find_element(By.XPATH, '//*[@id="jumpMenu"]')
    select = Select(dropdown_element)
    selected_option = select.first_selected_option

    print(f"Scraping page {page_number}...")

    if selected_option.text !=  'Ultimos Publicados (Más Recientes Primeros)':
        dropdown_element.click()
        time.sleep(2)
        ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
        ultimos_publicados.click()
        time.sleep(7)


    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        
        try:
            dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            
            # type validation, since a property can have two types
            try: 

                table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
                dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text

            except NoSuchElementException:
                dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text


            dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            
            if "MultipleSellers" in dictionary_cards['Link']:
                dictionary_cards['Broker'] = 'Multiple Sellers'

            else:

                # not all properties have brokers
                try:
                    dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards['Broker'] = 'No Broker'

         
            list_cards.append(dictionary_cards)
            
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        
        try:
            dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            
            # type validation, since a property can have two types
            try:
                table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
                dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text
            except NoSuchElementException:
                dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            
            dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')

            if "MultipleSellers" in dictionary_cards_2['Link']:
                dictionary_cards_2['Broker'] = 'Multiple Sellers'

            else:
                try:
                    dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards_2['Broker'] = 'No Broker'
    
            list_cards.append(dictionary_cards_2)
            
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
        page_number += 1
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)
cleaned_df = clean_real_estate_data(df)

In [ ]:
cleaned_df

In [93]:
cleaned_df.to_csv("classifieds_data.csv", index=False)

In [2]:
data = pd.read_csv('classifieds_data_v3.csv')

In [40]:
municipio = pd.read_csv('PR_Municipios')

In [ ]:
municipio

In [62]:
# Extract the part of the city column before the "-" sign
#data['city_name'] = data['City'].str.split('-').str[0].str.strip()
# Check if the word from the name column in municipio is present in the city column within data
data['city_name'] = data['City'].apply(lambda x: next((name for name in municipio['name'] if name in x), None))

# Merge the dataframes on the extracted city names and the name column
merged_df = pd.merge(data, municipio, left_on='city_name', right_on='name', how='left')

# Add the region column to the original data dataframe
data['region'] = merged_df['region']

# Drop the temporary city_name column
data.drop(columns=['city_name'], inplace=True)

In [51]:
data['Type'] = data['Type'].str.strip()

In [ ]:

data[data['region'].isna()]

In [ ]:
data['City'][85].split('-')[0]

In [3]:
data['Price'] = data['Price'].str.replace('$','').str.replace(',','').astype(float)

In [4]:
data

,Title,Rooms,Price,Type,Location,Link,Broker,Bedrooms,Bathrooms,Neighborhood,City,region
0,REPARTO ALEXANDRA,3 Cuartos | 3 Baños,379000.0,Casa,"Barrio-Algarrobo , Vega Baja",https://www.clasificadosonline.com/UDRealEstat...,Algarrobo de ANFE ADORNO,3,3,Barrio-Algarrobo,Vega Baja,north
1,Multi-Familiar de 2 Unidades en Bo. C...,>=6 Cuartos | 4 Baños,310000.0,MultiFamiliar,"Barrio-Collores , Humacao",https://www.clasificadosonline.com/UDRealEstat...,Collores de Orlando Rios,6,4,Barrio-Collores,Humacao,east
2,EXCELENTE PROPIEDAD BUEN PRECIO,2 Cuartos | 3 Baños,175000.0,Comercial,"Barrio-Pueblo , Aguas Buenas",https://www.clasificadosonline.com/UDRealEstat...,Pueblo de Batista Realty Group,2,3,Barrio-Pueblo,Aguas Buenas,east
3,BARRIO CANIACO EN UTUADO,2 Cuartos | 1 Baños,195000.0,Casa,"Barrio-Caniaco , Utuado",https://www.clasificadosonline.com/UDRealEstat...,Caniaco de ESPACIO VITAL PR PROPERTIES,2,1,Barrio-Caniaco,Utuado,north
4,"Residence, San Antonio, Dorado",4 Cuartos | 2 Baños,159000.0,Casa,"Barrio-San Antonio , Dorado",https://www.clasificadosonline.com/UDRealEstat...,San Antonio de RE/MAX Dorado Realty,4,2,Barrio-San Antonio,Dorado,north
...,...,...,...,...,...,...,...,...,...,...,...,...
85,O'Neil - Int. Milla de Oro & Roosevel...,>=6 Cuartos | 5 Baños,1350000.0,Comercial,"Sector-Hato Rey , San Juan - Hato Rey",https://www.clasificadosonline.com/UDRealEstat...,Hato Rey de AIRE,6,5,Sector-Hato Rey,San Juan - Hato Rey,metro
86,RENTAS 15 MIL. 4 locales 3222 p2,>=6 Cuartos | 5 Baños,625000.0,Comercial,"Barrio-Certenejas , Cidra",https://www.clasificadosonline.com/UDRealEstat...,Certenejas de 1RUIZ REALTY GROUP,6,5,Barrio-Certenejas,Cidra,east
87,NUEVO PRECIO APROVECHE AHORA,5 Cuartos | 2 Baños,139500.0,Comercial,"Barrio-Pueblo , Guayanilla",https://www.clasificadosonline.com/UDRealEstat...,Pueblo de TRANS GLOBAL REALTY GROUP,5,2,Barrio-Pueblo,Guayanilla,south
88,Exclusiva! Villa Caparra propiedad te...,4 Cuartos | 4 Baños,1300000.0,Casa,"Urbanizacion-Villa Caparra , Guaynabo",https://www.clasificadosonline.com/UDRealEstat...,Villa Caparra de Ani Fernandez Real Estate,4,4,Urbanizacion-Villa Caparra,Guaynabo,metro


In [5]:
data.to_csv("classifieds_data_v4.csv", index=False)

In [ ]:
data['Type'].unique()

In [63]:
data[data['Broker'] == 'No Broker']

,Title,Rooms,Price,Type,Location,Link,Broker,Bedrooms,Bathrooms,Neighborhood,City,region
44,NEGOCIO CON CLIENTELA OPERANDO EXITOS...,1 Baños,"$35,000",Comercial,"Urbanizacion-Sagrado Corazon , Guánica",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838405,No Broker,0,0,Urbanizacion-Sagrado Corazon,Guánica,south
52,Urb Alturas de la Placita,>= 6 Cuartos | 3 Baños,"$160,000",Casa,"Urbanizacion-Alturas De La Placita , Juncos",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838403,No Broker,6,3,Urbanizacion-Alturas De La Placita,Juncos,east
75,Solar 5 minutos de en Boquerón 1265 m...,NaN,"$88,000",Solar,"Barrio-Boqueron , Cabo Rojo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838396,No Broker,0,0,Barrio-Boqueron,Cabo Rojo,west
80,"UNICA residencia, Toa Alta - DtMF",5 Cuartos | > Baños,"$695,000",Casa,"Urbanizacion-Haciendas El Paraiso , Toa Alta",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838397,No Broker,0,0,Urbanizacion-Haciendas El Paraiso,Toa Alta,north
83,"Casa, Boquerón, 3 cuartos, 1 Baño 295...",3 Cuartos | 1 Baños,"$295,000",Casa,"Urbanizacion-Lumar , Cabo Rojo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838395,No Broker,3,1,Urbanizacion-Lumar,Cabo Rojo,west
